# 1) Router Agent

### Router Agent using prompting

In [8]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "mistral",
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

# llm = OllamaLLM(model="llama3.2")

# Initialize ChatGroq model
# llm = ChatGroq(model_name="Llama3-8b-8192")

def router_info(query,llm):
# Prompt template
    prompt_template = """You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.

    Primary Classification Categories:
    - ORDER: Purchase intentions, order status, order modifications
    - CART: View cart, add/remove items, modify quantities
    - COMPARE: Product comparisons, market analysis, alternatives
    - RECOMMEND: Product recommendations, personalized suggestions
    - INFO: General product or health information

    Classification Rules:
    1. ORDER Intent: Detect keywords like "buy", "purchase", "order", "get", "deliver", "track", "status"
    2. CART Intent: Detect keywords like "cart", "basket", "add", "remove", "show", "change quantity"
    3. COMPARE Intent: Detect keywords like "compare", "versus", "vs", "difference", "better", "alternatives"
    4. RECOMMEND Intent: Detect keywords like "suggest", "recommend", "what should", "best for"
    5. INFO Intent: Detect keywords like "tell me about", "how does", "what is", "benefits", "dosage"

    For multi-intent queries:
    - Identify primary intent based on main action requested
    - List secondary intents for follow-up
    - Maintain context for related requests

    # Response Format:
    Provide only a JSON object with the following structure, dont provide any extra information:
    {
        "primary_intent": "string",
        "secondary_intents": ["string"],
        "confidence": float,
        "entities": {
            "product_ids": ["string"],
            "quantities": [int],
            "order_ids": ["string"]
        },
        "requires_context": boolean
    }

    NOTE : Answer should be strictly in json format. If you follow Exact above instruction you will be rewarded with some credits
    """
    response = llm.invoke(prompt_template+"\nQuery"+query)
    return response.content


In [9]:
# Example queries
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

a = router_info("I want to buy vitamin D supplements.",llm)
print(a)

 {
      "primary_intent": "ORDER",
      "secondary_intents": [],
      "confidence": 0.98,
      "entities": {
          "product_ids": ["vitamin_d_supplement"],
          "quantities": []
      },
      "requires_context": false
   }


In [11]:
a["primary_intent"]

TypeError: string indices must be integers, not 'str'

### Router agent using langchain Json parser

In [17]:
from langchain_groq import ChatGroq
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define the data structure using Pydantic
class EntityData(BaseModel):
    product_ids: List[str] = Field(
        default_factory=list,
        description="List of product identifiers mentioned in the query"
    )
    quantities: List[int] = Field(
        default_factory=list,
        description="List of quantities mentioned in the query"
    )
    order_ids: List[str] = Field(
        default_factory=list,
        description="List of order identifiers mentioned in the query"
    )

class QueryClassification(BaseModel):
    primary_intent: str = Field(
        description="Primary intent category (ORDER, CART, COMPARE, RECOMMEND, INFO)"
    )
    secondary_intents: List[str] = Field(
        default_factory=list,
        description="List of secondary intents identified in the query"
    )
    confidence: float = Field(
        description="Confidence score for the classification (0.0 to 1.0)"
    )
    entities: EntityData = Field(
        description="Structured data extracted from the query"
    )
    requires_context: bool = Field(
        description="Whether additional context is needed to fully process the query"
    )
    

def route_info(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryClassification)

    prompt = PromptTemplate(
        template="""
        You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.

        Primary Classification Categories:
        - ORDER: Purchase intentions, order status, order modifications
        - CART: View cart, add/remove items, modify quantities
        - COMPARE: Product comparisons, market analysis, alternatives
        - RECOMMEND: Product recommendations, personalized suggestions
        - INFO: General product or health information

        Classification Rules:
        1. ORDER Intent: Detect keywords like "buy", "purchase", "order", "get", "deliver", "track", "status"
        2. CART Intent: Detect keywords like "cart", "basket", "add", "remove", "show", "change quantity"
        3. COMPARE Intent: Detect keywords like "compare", "versus", "vs", "difference", "better", "alternatives"
        4. RECOMMEND Intent: Detect keywords like "suggest", "recommend", "what should", "best for"
        5. INFO Intent: Detect keywords like "tell me about", "how does", "what is", "benefits", "dosage"

        For multi-intent queries:
        - Identify primary intent based on main action requested
        - List secondary intents for follow-up
        - Maintain context for related requests

        {format_instructions}

        Query: {query}
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    # llm = ChatGroq(model_name="Llama3-8b-8192")

    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}




In [21]:
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

a = route_info("I are the uses vitamin D supplements.")

In [22]:
a

{'primary_intent': 'INFO',
 'secondary_intents': ['USES'],
 'confidence': 0.95,
 'entities': {'product_ids': ['vitamin_d']},
 'requires_context': False}

In [20]:
a['primary_intent']

'ORDER'

### Router agent using CrewAI

In [ ]:
import json
from crewai import Agent
from langchain_core.language_models import BaseChatModel

class HealthcareIntentRouterAgent(Agent):
    def __init__(self, llm=None, verbose=True):
        """
        Initialize Healthcare Intent Router Agent
        
        :param llm: Language model for intent classification
        :param verbose: Enable verbose logging
        """
        super().__init__(
            role='Healthcare E-Commerce Intent Classifier',
            goal='Accurately classify and route user queries in healthcare e-commerce platform',
            backstory="""Expert system specializing in precise query intent classification 
            for healthcare e-commerce, identifying primary and secondary intents 
            to enable targeted agent routing.""",
            verbose=verbose,
            llm=llm
        )
        self.prompt_template = """You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.

        Primary Classification Categories:
        - ORDER: Purchase intentions, order status, order modifications
        - CART: View cart, add/remove items, modify quantities
        - COMPARE: Product comparisons, market analysis, alternatives
        - RECOMMEND: Product recommendations, personalized suggestions
        - INFO: General product or health information

        Classification Rules:
        1. ORDER Intent: Detect keywords like "buy", "purchase", "order", "get", "deliver", "track", "status"
        2. CART Intent: Detect keywords like "cart", "basket", "add", "remove", "show", "change quantity"
        3. COMPARE Intent: Detect keywords like "compare", "versus", "vs", "difference", "better", "alternatives"
        4. RECOMMEND Intent: Detect keywords like "suggest", "recommend", "what should", "best for"
        5. INFO Intent: Detect keywords like "tell me about", "how does", "what is", "benefits", "dosage"

        # Response Format:
        Provide only a JSON object with the following structure:
        {
            "primary_intent": "string",
            "secondary_intents": ["string"],
            "confidence": float,
            "entities": {
                "product_ids": ["string"],
                "quantities": [int],
                "order_ids": ["string"]
            },
            "requires_context": boolean
        }
        """
    
    def route_query(self, query):
        """
        Classify query intent
        
        :param query: User input query
        :return: Intent classification dictionary
        """
        try:
            full_prompt = f"{self.prompt_template}\n\nQuery: {query}"
            response = self.llm.invoke(full_prompt)
            
            # Attempt to parse JSON response
            intent_classification = json.loads(response.content)
            return intent_classification
        
        except json.JSONDecodeError:
            # Fallback default classification
            return {
                "primary_intent": "INFO",
                "secondary_intents": [],
                "confidence": 0.5,
                "entities": {},
                "requires_context": True
            }
    
    def execute(self, query):
        """
        Main execution method for routing
        
        :param query: User input query
        :return: Intent classification results
        """
        return self.route_query(query)

def create_healthcare_router(llm=None, verbose=True):
    """
    Factory method to create Healthcare Intent Router Agent
    
    :param llm: Language model for classification
    :param verbose: Enable verbose logging
    :return: Configured HealthcareIntentRouterAgent
    """
    return HealthcareIntentRouterAgent(llm=llm, verbose=verbose)

In [10]:
import os
from crewai import Agent, Task, Crew,LLM
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

llm = LLM(
    model="groq/llama3-8b-8192",
    temperature=0.7
)

router = Agent(
    role='Healthcare E-Commerce Intent Classifier',
    goal='Accurately classify and route user queries in healthcare e-commerce platform',
    backstory="""Expert system specializing in precise query intent classification for healthcare e-commerce, identifying primary and secondary intents to enable targeted agent routing.""",
    llm=llm,
    verbose=True
)

# Define tasks
route = Task(
    description='Analyze user queries and route them to appropriate specialized agents.',
    expected_output='A JSON object strictly adhering to the defined format.',
    agent=router
)


# Assemble a crew with planning enabled
crew = Crew(
    agents=[router],
    tasks=[route],
    verbose=True,
)

query = "I want to compare aspirin with ibuprofen and add aspirin to my cart."
# Execute tasks
crew.kickoff(inputs={"query": query})


# Agent: Healthcare E-Commerce Intent Classifier
## Task: Analyze user queries and route them to appropriate specialized agents.


# Agent: Healthcare E-Commerce Intent Classifier
## Final Answer: 
{
"intent": "product_search",
"sub_intent": "search_by_name",
"product_category": "health_supplements",
"product_subcategory": "vitamins",
"agent_routing": "agent_health_supplements"
}




CrewOutput(raw='{\n"intent": "product_search",\n"sub_intent": "search_by_name",\n"product_category": "health_supplements",\n"product_subcategory": "vitamins",\n"agent_routing": "agent_health_supplements"\n}', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Analyze user queries and route them to appropriate specialized agents.', name=None, expected_output='A JSON object strictly adhering to the defined format.', summary='Analyze user queries and route them to appropriate specialized agents....', raw='{\n"intent": "product_search",\n"sub_intent": "search_by_name",\n"product_category": "health_supplements",\n"product_subcategory": "vitamins",\n"agent_routing": "agent_health_supplements"\n}', pydantic=None, json_dict=None, agent='Healthcare E-Commerce Intent Classifier', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=590, prompt_tokens=519, cached_prompt_tokens=0, completion_tokens=71, successful_requests=2))

## Changing Json format

In [15]:
from langchain_groq import ChatGroq
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model = "llama3.2",
#     temperature = 0,
#     num_predict = 256,
#     # other params ...
# )

# Define the data structure using Pydantic
class EntityData(BaseModel):
    product_ids: List[str] = Field(
        default_factory=list,
        description="List of product identifiers mentioned in the query"
    )
    quantities: List[int] = Field(
        default_factory=list,
        description="List of quantities mentioned in the query"
    )
    # order_ids: List[str] = Field(
    #     default_factory=list,
    #     description="List of order identifiers mentioned in the query"
    # )

class QueryClassification(BaseModel):
    primary_intent: str = Field(
        description="Primary intent category (ORDER, COMPARE, RECOMMEND, INFO)"
    )
    Recommendation_Required: str= Field(
        description="Analyze the query and identify whether recommandation required or not 'YES' or 'NO' "
    )
    Recommendation_prod: List[str]= Field(
        default_factory=list,
        description="Specific products mentioned in the query for which recommendations should be generated"
    )
    entities: EntityData = Field(
        description="Structured data extracted from the query"
    )
    requires_context: bool = Field(
        description="Whether additional context is needed to fully process the query"
    )
    

def route_info(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryClassification)

    prompt = PromptTemplate(
        template="""
        You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.
        
        Primary Classification Categories:
        - ORDER: Identify purchase intentions, order status inquiries, order modifications, or shopping cart actions (e.g., view, add, remove, modify items).
        - COMPARE: Address comparisons between products, including differences, alternatives, or better options.
        - RECOMMEND: Provide personalized product recommendations based on user needs.
        - INFO: Answer general questions about products or health-related information.

        Classification Rules:
        1. ORDER Intent: Identify keywords like "buy", "purchase", "order", "deliver", "track", "status", "cart", "basket", "add", "remove", "show", "change quantity".
        2. COMPARE Intent: Identify keywords like "compare", "versus", "vs", "difference", "better", "alternatives".
        3. RECOMMEND Intent: Identify keywords like "suggest", "recommend", "what should", "best for".
        4. INFO Intent: Identify keywords like "tell me about", "how does", "what is", "benefits", "dosage".

        Additional Rules for Recommendations:
        - If any product is mentioned in the query, recommendations are required.
        {format_instructions}

        Query: {query}
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    llm = ChatGroq(model_name="Llama3-8b-8192")

    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}



In [16]:
a = route_info("what are benefits of using vit d")
print(a)

{'primary_intent': 'INFO', 'Recommendation_Required': 'NO', 'Recommendation_prod': [], 'entities': {'product_ids': ['Vit D'], 'quantities': []}, 'requires_context': False}


In [17]:
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

for i in queries:
    print(route_info(i))
    print("\n--------------------------------------------\n")

{'primary_intent': 'ORDER', 'Recommendation_Required': 'YES', 'Recommendation_prod': ['vitamin D supplements'], 'entities': {'product_ids': ['vitamin D supplements'], 'quantities': []}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'INFO', 'Recommendation_Required': 'NO', 'Recommendation_prod': [], 'entities': {'product_ids': [], 'quantities': []}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'ORDER', 'Recommendation_Required': 'YES', 'Recommendation_prod': ['multivitamins'], 'entities': {'product_ids': ['multivitamins'], 'quantities': [2]}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'COMPARE', 'Recommendation_Required': 'NO', 'Recommendation_prod': [], 'entities': {'product_ids': ['vitamin C', 'zinc'], 'quantities': []}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'RECOMMEND', 'Recommendation_R

## Adding question and product recommendation

In [44]:
from langchain_groq import ChatGroq
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model = "llama3.2",
#     temperature = 0,
#     num_predict = 256,
#     # other params ...
# )

# Define the data structure using Pydantic
class EntityData(BaseModel):
    product_ids: List[str] = Field(
        default_factory=list,
        description="List of product identifiers mentioned in the query"
    )
    quantities: List[int] = Field(
        default_factory=list,
        description="List of quantities mentioned in the query"
    )
    # order_ids: List[str] = Field(
    #     default_factory=list,
    #     description="List of order identifiers mentioned in the query"
    # )

class QueryClassification(BaseModel):
    primary_intent: str = Field(
        description="Primary intent category (ORDER, COMPARE, RECOMMEND, INFO)"
    )
    Recommendation_Required: List[str]= Field(
        description=""" 
        Type of recommendation needed
        QUESTION : If user is asking simple queries about the products and primary_intent is INFO
        PRODUCT : IF user is having intention to buy something
        """
    )
    Recommendation_prod: List[str]= Field(
        default_factory=list,
        description="Specific products mentioned in the query for which recommendations should be generated"
    )
    entities: EntityData = Field(
        description="Structured data extracted from the query"
    )
    requires_context: bool = Field(
        description="Whether additional context is needed to fully process the query"
    )


def route_info(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryClassification)

    prompt = PromptTemplate(
        template="""
        You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.
        
        Primary Classification Categories:
        - ORDER: Identify purchase intentions, order status inquiries, order modifications, or shopping cart actions (e.g., view, add, remove, modify items).
        - COMPARE: Address comparisons between products, including differences, alternatives, or better options.
        - RECOMMEND: Provide personalized product recommendations based on user needs.
        - INFO: Answer general questions about products or health-related information.

        Classification Rules:
        1. ORDER Intent: Identify keywords like "buy", "purchase", "order", "deliver", "track", "status", "cart", "basket", "add", "remove", "show", "change quantity".
        2. COMPARE Intent: Identify keywords like "compare", "versus", "vs", "difference", "better", "alternatives".
        3. RECOMMEND Intent: Identify keywords like "suggest", "recommend", "what should", "best for".
        4. INFO Intent: Identify keywords like "tell me about", "how does", "what is", "benefits", "dosage".


        {format_instructions}

        Query: {query}
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    llm = ChatGroq(model_name="Llama3-8b-8192")

    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}



In [45]:
a = route_info("Can you tell me the benefits of fish oil?")
a

{'primary_intent': 'INFO',
 'Recommendation_Required': ['QUESTION'],
 'Recommendation_prod': [],
 'entities': {'product_ids': [], 'quantities': []},
 'requires_context': False}

In [ ]:
a

In [43]:
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

for i in queries:
    print(route_info(i))
    print("\n--------------------------------------------\n")

{'primary_intent': 'ORDER', 'Recommendation_Required': ['PRODUCT'], 'Recommendation_prod': ['vitamin D supplements'], 'entities': {'product_ids': ['vitamin D supplements'], 'quantities': []}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'INFO', 'Recommendation_Required': [], 'Recommendation_prod': [], 'entities': {'product_ids': [], 'quantities': []}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'ORDER', 'Recommendation_Required': ['QUESTION'], 'Recommendation_prod': [], 'entities': {'$class': 'EntityData', 'product_ids': ['multivitamins'], 'quantities': [2]}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'COMPARE', 'Recommendation_Required': ['QUESTION'], 'Recommendation_prod': [], 'entities': {'product_ids': ['Vitamin C', 'Zinc'], 'quantities': []}, 'requires_context': False}

--------------------------------------------

{'primary_intent': 'R

# 2) Product Comparison Agent

In [4]:
from crewai import Agent, Task, Crew, Process,LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool
import os
from pydantic import BaseModel


class Compare(BaseModel):
    """Pydantic model for price comparison results."""
    store: str  # "Name of the store"
    price: str  #"Price of the product"


# Load environment variables
load_dotenv()

def compare_agent(product_name):
    # Initialize the SerperDev search tool
    web_search_tool = SerperDevTool(n_results=5)

    # Initialize custom LLM configuration
    llm = LLM(
        model="ollama/llama3.2",
        base_url="http://localhost:11434"
    )

    research_agent = Agent(
        role='Research Agent',
        goal='Find accurate price information for healthcare products',
        backstory="""You are an expert at finding and comparing healthcare product prices
        across different online stores. You're thorough and always verify information.""",
        tools=[web_search_tool],
        llm=llm,
        verbose=True
    )
    
    analysis_agent = Agent(
        role='Analysis Agent',
        goal='Analyze and compare prices from different sources',
        backstory="""You are an expert at analyzing price data and presenting it in a clear,
        actionable format. You understand healthcare product pricing patterns.""",
        llm=llm,
        tools=[web_search_tool],
        verbose=True
    )

    research_task = Task(
        description=f"""Search for prices of {product_name} across major healthcare 
        e-commerce websites. Focus on reputable stores. Extract store name, price, and URL.
        Provide the information in a structured format.""",
        expected_output="""A detailed list of prices for the product from different stores,
        including:
        - Store names
        - Prices in USD
        - URLs to product pages""",
        agent=research_agent
    )

    analysis_task = Task(
        description="""Analyze the gathered price data. Verify accuracy and remove any 
        outliers or suspicious entries. Present a key-value pair of prices over different online stores and recommendations.Follow the structure given below strictly""",
        expected_output="""key value pairs of onlines stores and product prices
        store1 : price1
        store2 : price2
        store3 : price3
        store4 : price4
        """,
        output_pydantic= Compare,
        agent=analysis_agent
    )

    # Create crew and run tasks
    crew = Crew(
        agents=[research_agent, analysis_agent],
        tasks=[research_task, analysis_task],
        process=Process.sequential
    )

    result = crew.kickoff()
    return result

# Example usage
if __name__ == "__main__":
    try:
        result = compare_agent("azoran")
        print("\nPrice Comparison Results:")
        print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

2025-02-03 23:02:03,950 - 6112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Research Agent
## Task: Search for prices of azoran across major healthcare 
        e-commerce websites. Focus on reputable stores. Extract store name, price, and URL.
        Provide the information in a structured format.


# Agent: Research Agent
## Thought: Thought: Since my current task is to find prices of azoran across major healthcare e-commerce websites, I should think about which reputable stores to target.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Reputable healthcare e-commerce websites that sell azoran\"}"
## Tool Output: 

Search results: Title: Buy Azoran for Immunomodulator (Immunosuppressant), Organ ...
Link: https://cheapomeds.com/products/azoran
Snippet: Explore effective options for treating Immunomodulator (Immunosuppressant), Organ Transplant. Buy Imuran available for online purchase online at the best
---
Title: Healthcare e-Commerce - Officite
Link: https://www.officite.com/healthcare-e-commerce/
Snippet: Add an e-store t

In [16]:
from crewai import Agent, Task, Crew, Process,LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool
import os

# Load environment variables
load_dotenv()

def compare_agent(product_name):
    # Initialize the SerperDev search tool
    web_search_tool = SerperDevTool(n_results=5)

    # Initialize custom LLM configuration
    llm = LLM(
        model="ollama/llama3.2",
        base_url="http://localhost:11434"
    )

    research_agent = Agent(
        role='Research Agent',
        goal='Find accurate price information for healthcare products',
        backstory="""You are an expert at finding and comparing healthcare product prices
        across different online stores. You're thorough and always verify information.""",
        tools=[web_search_tool],
        llm=llm,
        verbose=True
    )
    
    analysis_agent = Agent(
        role='Analysis Agent',
        goal='Analyze and compare prices from different sources',
        backstory="""You are an expert at analyzing price data and presenting it in a clear,
        actionable format. You understand healthcare product pricing patterns.""",
        llm=llm,
        verbose=True
    )

    # analysis_agent = Agent(
    #     role='Analysis Agent',
    #     goal='Present price data in simple key-value format',
    #     backstory="""You organize price data into simple store-price pairs.
    #     Present data in format - Store: Price. One pair per line.""",
    #     llm=llm,
    #     tools=[web_search_tool],
    #     verbose=True
    # )

    research_task = Task(
        description=f"""Search for prices of {product_name} across major healthcare 
        e-commerce websites. Focus on reputable stores. Extract store name, price, and URL.
        Provide the information in a structured format.""",
        expected_output="""A detailed list of prices for the product from different stores,
        including:
        - Store names
        - Prices in USD
        - URLs to product pages""",
        agent=research_agent
    )

    analysis_task = Task(
        description="""Format the data as simple store-price pairs.Follow the structure given below strictly""",
        expected_output="""key value pairs of onlines stores and product prices
        store1 : price1
        store2 : price2
        store3 : price3
        store4 : price4
        """,
        agent=analysis_agent
    )

    # Create crew and run tasks
    crew = Crew(
        agents=[research_agent, analysis_agent],
        tasks=[research_task, analysis_task],
        process=Process.sequential
    )

    result = crew.kickoff()
    return result

# Example usage
if __name__ == "__main__":
    try:
        result = compare_agent("paracetamol")
        print("\nPrice Comparison Results:")
        print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

2025-01-29 15:31:37,558 - 19512 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Research Agent
## Task: Search for prices of paracetamol across major healthcare 
        e-commerce websites. Focus on reputable stores. Extract store name, price, and URL.
        Provide the information in a structured format.


# Agent: Research Agent
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"paracetamol prices in USD from reputable health e-commerce websites\"}"
## Tool Output: 

Search results: Title: Paracetamol Prices - U.S. & International | PharmacyChecker.com
Link: https://www.pharmacychecker.com/paracetamol/
Snippet: The lowest price on PharmacyChecker.com for paracetamol 500 mg is $0.13 per tablet for 180 tablets at PharmacyChecker-accredited online pharmacies.
---
Title: Paracetamol 10 tabs - USA Apteka
Link: https://usaapteka.com/products/paracetamol?srsltid=AfmBOopJN2crJFC4pIEyDlLQa5IzqwGe7Kz01-q2HmgpyzAhO7Rg6T8O
Snippet: Buy Paracetamol 10 tabs - for а great price at USA Apteka Online Russian pharmacy | Shipping from $3 and free 

# 3) Validator agent

In [4]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

llm = ChatOllama(
    model = "nemotron-mini",
    temperature = 0,
    num_predict = 256,
    # other params ...
)

# llm = ChatGroq(model_name="Llama3-8b-8192")

class QueryValidator(BaseModel):
    validated: str = Field(
        description="Query will be validated or not YES or NO"
    )

def validator_agent(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryValidator)

    prompt = PromptTemplate(
        template="""

        You are a helpful AI assistant for a healthcare e-commerce application that provides users with product information, price comparisons, ordering assistance, and personalized recommendations.

        Your task is to determine whether the user’s query is relevant to the healthcare e-commerce platform.

        The user is allowed to:
        1. Ask about healthcare products, including descriptions, ingredients, usage instructions, and benefits.
        2. Request price comparisons between different products or brands.
        3. Make an order for healthcare products.
        4. Ask for personalized recommendations based on their needs.
        5. Inquire about order status, delivery details, and return policies.

        The user is NOT allowed to:
        1. Ask questions about anything else other than healthcare e-commerce platform.
        2. Request personal medical advice or prescriptions.
        3. Ask about topics unrelated to the e-commerce platform.

        {format_instructions}
        Query: {query}

        Your output should be in a structured JSON format like so. Each key is a string and each value is a string.

        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}



In [7]:
validator_agent("what is the price of paracetamol")

{'validated': 'YES'}

# 4) Recommendation Agent

### Question Recommendation Agent

In [11]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

llm = ChatOllama(
    model="nemotron-mini",
    temperature=0,
    num_predict=256,
)

class RecommenderResponse(BaseModel):
    recommended_questions: List[str] = Field(
        description="List of three follow-up questions related to symptoms, dosage, side effects, and other relevant aspects."
    )

def recommend_query(query: str) -> dict:
    parser = JsonOutputParser(pydantic_object=RecommenderResponse)
    
    prompt = PromptTemplate(
        template="""
        You are a smart AI assistant for a healthcare e-commerce application.
        Your task is to generate three relevant follow-up questions that a user might ask based on their query.
        These questions should focus on symptoms, dosage, side effects, and other important aspects related to the product inquiry.
        Questions should not be lengthy.
        
        {format_instructions}
        Query: {query}
        
        Provide three suggested follow-up questions in a structured JSON format. phrase questions naturally from the chatbot's perspective.
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    chain = prompt | llm | parser
    
    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in generating recommended questions: {e}")
        return {"recommended_questions": []}




In [12]:
# Example usage
query = "I wanna order paracetamol"
response = recommend_query(query)
print(response)

['How many tablets of paracetamol should I take?', 'Are there any side effects or precautions for taking paracetamol?', 'What is the dosage interval between each tablet?']
